# yes24 데이터 수집

---

## 목표

<b>[과제 설명]</b>

yes24(http://www.yes24.com/) 사이트의 베스트셀러 페이지에서<br>
국내도서 베스트셀러 순위 1-10위 책에 대해<br>
<b>순위, 제목, 저자, 출판사, 판매가, 출간일</b>을 프린트하시오.<br>
-----
<b>[출력 예시]</b><br>    
1. 제목: 트렌드 코리아 2024, 저자: 김난도, 전미영, 최지혜, 이수진, 권정윤 외 6명, 출판사: 미래의창, 판매가: 17,100원, 출간일: 2023년 10월 05일<br>
2. ........................<br>
3. ........................<br>
4. ....................<br>
5. ...................<br>

## selenium 설치

In [1]:
# !pip install selenium

## 베스트 셀러 페이지(html) 읽어오기

In [2]:
import requests

base_url = 'https://www.yes24.com'
bestseller_url = '/Product/Category/BestSeller'
URL = base_url + bestseller_url
main_html = requests.get(URL)

## html 파싱하기

- 파서: lxml로 설정

In [3]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(main_html.text, 'lxml')

### 도서 상세 페이지 URL 파싱

In [4]:
a_list = soup.find_all('a', {'class': 'lnk_img'})

### 도서 상세 페이지 읽어오기 & 정보 파싱하여 출력하기

In [5]:
# 조건에 맞게 저자 이름 파싱
def getWriterName(writerInfo):
    splited_writer = writerInfo.split('/')
    writer = splited_writer[0]
    list = [' 저', ' 원저', ' 글', ' 편']
    for word in list:
        writer = writer.replace(word, '')
    return writer

# 가격 정보를 파싱하는 함수
def getPrice(soup):
    price_selector_list = ['#yDetailTopWrap > div.topColRgt > div.gd_infoBot > div.gd_infoTbArea > \
                            div:nth-child(4) > table > tbody > tr:nth-child(2) > td > span > em',
                           '#yDetailTopWrap > div.topColRgt > div.gd_infoBot > div.gd_infoTbArea > \
                           div:nth-child(3) > table > tbody > tr:nth-child(2) > td > span > em']
        
    for selector in price_selector_list:
        price = soup.select(selector)
        if len(price) > 0:
            return price[0].text
    return '0'

# 책정보(제목, 저자, 출판사, 판매가, 출간일)를 파싱하는 함수
def getBookInfo(book_html):
    soup_book = BeautifulSoup(book_html.text, 'lxml')
    # 제목
    title = soup_book.find('h2', {'class': 'gd_name'}).text.strip()
    
    # 저자
    writerInfo = soup_book.find('span', {'class': 'gd_auth'}).text.strip().split('\n')[0].strip()
    writer = getWriterName(writerInfo)
    
    # 출판사
    publisher = soup_book.find('span', {'class': 'gd_pub'}).text.strip()
    
    # 판매가
    price = getPrice(soup_book)
    
    # 출간일
    date = soup_book.find('span', {'class': 'gd_date'}).text.strip()
    book_info = ('제목: {0}, 저자: {1}, 출판사: {2}, 판매가: {3}원, 출간일: {4}').format(title, writer, publisher, price, date)
    return book_info

In [6]:
# 국내도서 베스트셀러 10위 정보 출력
num = 10

for index in range(num):
    # 도서 상세 페이지 읽어오기
    href = a_list[index].get('href')
    book_url = base_url + href
    book_html = requests.get(book_url)

    # index+1번째 책의 상세정보 파싱하여 출력하기
    result = getBookInfo(book_html)
    index = index + 1
    print(str(index) + '.' + result)    

1.제목: 퓨처 셀프, 저자: 벤저민 하디, 출판사: 상상스퀘어, 판매가: 17,820원, 출간일: 2023년 08월 30일
2.제목: 트렌드 코리아 2024, 저자: 김난도, 전미영, 최지혜, 이수진, 권정윤 외 6명, 출판사: 미래의창, 판매가: 17,100원, 출간일: 2023년 10월 05일
3.제목: 시대예보: 핵개인의 시대, 저자: 송길영, 출판사: 교보문고, 판매가: 18,900원, 출간일: 2023년 09월 25일
4.제목: 세이노의 가르침, 저자: 세이노(SayNo), 출판사: 데이원, 판매가: 6,480원, 출간일: 2023년 03월 02일
5.제목: 도시와 그 불확실한 벽, 저자: 무라카미 하루키, 출판사: 문학동네, 판매가: 17,550원, 출간일: 2023년 09월 06일
6.제목: 파견자들, 저자: 김초엽, 출판사: 퍼블리온, 판매가: 17,100원, 출간일: 2023년 10월 13일
7.제목: 도둑맞은 집중력, 저자: 요한 하리, 출판사: 어크로스, 판매가: 16,920원, 출간일: 2023년 04월 28일
8.제목: 2024 틴틴팅클 애착일력, 저자: 난, 출판사: 위즈덤하우스, 판매가: 21,600원, 출간일: 2023년 11월 08일
9.제목: 올웨더 투자법, 저자: 판교불패, 출판사: 알에이치코리아(RHK), 판매가: 19,800원, 출간일: 2023년 10월 18일
10.제목: 흔한남매 14, 저자: 흔한남매, 출판사: 미래엔아이세움, 판매가: 13,050원, 출간일: 2023년 09월 21일
